# Introduction

In [1]:
from ipywidgets import interact
from upsetjs_jupyter_widget import UpSetWidget

In [2]:
w = UpSetWidget[str]()
w.theme = 'dark'
w.mode = 'hover'

In [3]:
w.from_list(sets=dict(one = ['a', 'b', 'c', 'e', 'g', 'h', 'k', 'l', 'm'], two = ['a', 'b', 'd', 'e', 'j'], three = ['a', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm']))
pass

In [4]:
w.elems

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm']

In [5]:
w.combinations

[UpSetSetIntersection(name=one, sets={'one'}, elems={'k', 'e', 'h', 'b', 'g', 'c', 'l', 'a', 'm'}),
 UpSetSetIntersection(name=three, sets={'three'}, elems={'j', 'e', 'h', 'g', 'l', 'a', 'm', 'i', 'f'}),
 UpSetSetIntersection(name=(one ∩ three), sets={'one', 'three'}, elems={'g', 'l', 'a', 'm', 'e', 'h'}),
 UpSetSetIntersection(name=two, sets={'two'}, elems={'j', 'a', 'd', 'e', 'b'}),
 UpSetSetIntersection(name=(one ∩ two), sets={'one', 'two'}, elems={'e', 'a', 'b'}),
 UpSetSetIntersection(name=(three ∩ two), sets={'two', 'three'}, elems={'j', 'e', 'a'})]

In [6]:
w

UpSetWidget(value=None, elems=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm'], height_ratios…

In [7]:
def selection_changed(s, ws):
    # w.selection = ws["name"] if ws else None
    return s, ws # s["name"] if s else None
interact(selection_changed, s=10, ws=w)

interactive(children=(IntSlider(value=10, description='s', max=30, min=-10), UpSetWidget(value=None, elems=['a…

<function __main__.selection_changed(s, ws)>